In [1]:
from src.dysregnet import dysregnet
import pandas as pd
import pickle
import numpy as np
import os

### GTEx Data

The tissue-specific Gene Regulatory Networks (GRNs) are generated using GENIE3 with the GTEx expression data. 

Gene expression data from the GTEx data base:

In [2]:
current_dir = os.getcwd()
exp_lung = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv(current_dir + "/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")

In [3]:
grn_lung = pd.read_csv(current_dir + "/GENIE3_output/linkedList_output_slurm_gene_tpm_v10_lung_filtered.csv")


Generate meta data for the GTEx data

In [4]:
meta_breast = pd.DataFrame({
    'sample': exp_breast.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_breast.columns[1:])),
    'gender' : np.random.choice(["MALE", "FEMALE"], size=len(exp_breast.columns[1:]))
})
meta_breast.head(5)

,sample,condition,gender
0,GTEX-1117F-2826-SM-5GZXL,1,FEMALE
1,GTEX-111YS-1926-SM-5GICC,0,FEMALE
2,GTEX-1122O-1226-SM-5H113,1,MALE
3,GTEX-117XS-1926-SM-5GICO,1,FEMALE
4,GTEX-117YX-1426-SM-5H12H,1,FEMALE


In [5]:
meta_lung= pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([1], size=len(exp_lung.columns[1:])),
    'gender' : np.random.choice(["MALE", "FEMALE"], size=len(exp_lung.columns[1:]))
})
meta_lung.iloc[0,1] = 0
meta_lung.iloc[1,1] = 0
meta_lung.iloc[2,1] = 0
meta_lung.iloc[3,1] = 0
meta_lung.iloc[4,1] = 0
meta_lung.head(5)


,sample,condition,gender
0,GTEX-111CU-0326-SM-5GZXO,0,MALE
1,GTEX-111FC-1126-SM-5GZWU,0,FEMALE
2,GTEX-111VG-0726-SM-5GIDC,0,MALE
3,GTEX-111YS-0626-SM-5GZXV,0,MALE
4,GTEX-1122O-0126-SM-5GICA,0,MALE


In [6]:
exp_lung = exp_lung.set_index(exp_lung.columns[0])
exp_lung = exp_lung.T 
exp_lung.insert(0, "sample", exp_lung.index)
exp_lung.head(1)

Description,sample,WASH7P,OR4F5,CICP27,WASH9P,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB
GTEX-111CU-0326-SM-5GZXO,GTEX-111CU-0326-SM-5GZXO,1.73487,0.015715,0.404045,3.44562,13.6032,701.845,36.4011,7.73125,21.465,...,20639.6,30316.0,19533.3,9311.24,11979.6,25032.7,9814.89,13675.4,41.7375,17064.5


In [7]:
exp_breast = exp_breast.set_index(exp_breast.columns[0])
exp_breast = exp_breast.T 
exp_breast.insert(0, "sample", exp_breast.index)
exp_breast.head(1)

Description,sample,WASH7P,OR4F5,WASH9P,MTND1P23,MTND2P28,MTCO1P12,MTCO2P12,MTATP8P1,MTATP6P1,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-TE,MT-CYB,MT-TP
GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2826-SM-5GZXL,9.70327,0.018298,7.16661,18.5431,1017.12,38.0448,10.6094,20.5299,4373.92,...,41003.6,51020.1,14202.3,14019.8,35322.9,2929.78,1756.72,2.08274,20157.3,2.11337


In [8]:
# Step 1: Filter rows where condition is 0 (controls)
controls = meta_breast[meta_breast['condition'] == 0]

# Step 2: Randomly select two control samples
selected_controls = controls.sample(n=2, random_state=42)  # random_state for reproducibility

# Step 3: Update the meta_breast DataFrame to only include the selected controls and all other rows where condition != 0
meta_breast_filtered = pd.concat([
    selected_controls,
    meta_breast[meta_breast['condition'] != 0]
], ignore_index=True)

# Display the updated meta_breast DataFrame
print(meta_breast_filtered)


                       sample  condition  gender
0    GTEX-18A6Q-0926-SM-7LG4N          0  FEMALE
1     GTEX-YEC3-1026-SM-5IFI5          0  FEMALE
2    GTEX-1117F-2826-SM-5GZXL          1  FEMALE
3    GTEX-1122O-1226-SM-5H113          1    MALE
4    GTEX-117XS-1926-SM-5GICO          1  FEMALE
..                        ...        ...     ...
276   GTEX-ZVT2-1826-SM-5NQ8W          1    MALE
277   GTEX-ZVT4-1026-SM-57WC4          1  FEMALE
278   GTEX-ZYFC-0826-SM-5E44K          1  FEMALE
279   GTEX-ZYY3-2526-SM-GMXAZ          1    MALE
280   GTEX-ZZPU-0626-SM-5E43T          1    MALE

[281 rows x 3 columns]


Define the confounding variables or the design matrix

In [9]:
# Define condition column (0 indicated control, 1 indicates case)
# Column name for the condition in the meta DataFrame.
conCol='condition'

# Define categorical confounder columns in meta dataframe, List of categorical variable names. 
# They should match the name of their columns in the meta Dataframe.
CatCov=['gender']  

### Run DysRegNet
You can run dysregnet by passing a GRN or a tissue type. In case a tissue type is provided, the tissue specific GRN and correspondign linear regression models will be downloaded locally. 

In [10]:
data=dysregnet.run(expression_data = exp_lung.head(14), 
                   GRN= "lung",
                   meta = meta_lung.head(15),
                   direction_condition=False,
                   normaltest=True,
                   conCol=conCol,
                   #CatCov=CatCov,
                   R2_threshold=.2,
                   skip_poor_fits = True)

Processing lung GRN file...
GRN file and model ZIP already exist locally. Skipping download.
Extracting lung model ZIP...
Extracted models/lung_models.zip to models
You did not input any covariates in CatCov or ConCov parameters, proceeding without them.


Processing edges: 100000it [02:28, 674.20it/s]


Ratio of found models:  1.0
Skipped models:  0.0


In [11]:
data.get_results()

,"(ZFY, TTTY14)","(ZFY, EIF1AY)","(ZFY, UTY)","(ZFY, TXLNGY)","(ZFY, USP9Y)","(ZFY, KDM5D)","(ZFY, PRKY)","(ZFY, RPS4Y1)","(ZFY, DDX3Y)","(ZFY, NLGN4Y)",...,"(ZKSCAN4, HNRNPA1L2)","(BCL6, CSF3R)","(SP3, DYNC1LI2)","(MYSM1, ZNF714)","(SOX2, OR7E36P)","(ZNF571, PRPSAP1)","(L3MBTL1, CERS3-AS1)","(TWIST1, DIO2)","(DMRTA1, MAGI3)","(THAP12, TAF9B)"
patient id,,,,,,,,,,,,,,,,,,,,,
GTEX-1128S-0726-SM-5N9D6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-117YW-0526-SM-5H11C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-117YX-1326-SM-5H125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11DXX-0626-SM-5Q5AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11DXZ-0726-SM-5N9C4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11DYG-0626-SM-GINAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11DZ1-0426-SM-5H11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11EI6-0826-SM-5985V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GTEX-11EMC-0126-SM-5EGKV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# get the patient-specific dysregulate networks
results = data.get_results()

# or with binary edges
results = data.get_results_binary()

# get R2 values, coefficients, and coefficient p-values for all models/edges
data.get_model_stats()

,R2,coef_intercept,coef_TF,pval_intercept,pval_TF
"(ZFY, TTTY14)",0.459116,0.271082,0.114345,2.291874e-26,2.210604e-82
"(ZFY, EIF1AY)",0.628702,3.399538,1.715984,6.250001e-35,1.255384e-131
"(ZFY, UTY)",0.405454,2.845315,0.968856,1.646112e-31,5.454703e-70
"(ZFY, TXLNGY)",0.648989,3.002524,1.790237,2.208948e-28,5.584703e-139
"(ZFY, USP9Y)",0.447088,3.008498,1.117954,2.156211e-31,1.680314e-79
...,...,...,...,...,...
"(ZNF571, PRPSAP1)",0.423250,11.086366,7.599814,9.008516e-29,5.686712e-74
"(L3MBTL1, CERS3-AS1)",0.311288,-0.140535,0.201972,5.070439e-02,1.030897e-50
"(TWIST1, DIO2)",0.359539,0.696419,0.453383,1.345792e-20,3.069666e-60
"(DMRTA1, MAGI3)",0.375117,5.846190,5.263269,8.333980e-55,1.816033e-63


In [13]:
results
edge_counts = results.sum()
edge_counts

(ZFY, TTTY14)           0.0
(ZFY, EIF1AY)           0.0
(ZFY, UTY)              0.0
(ZFY, TXLNGY)           0.0
(ZFY, USP9Y)            0.0
                       ... 
(ZNF571, PRPSAP1)       0.0
(L3MBTL1, CERS3-AS1)    0.0
(TWIST1, DIO2)          0.0
(DMRTA1, MAGI3)         0.0
(THAP12, TAF9B)         0.0
Length: 100000, dtype: float64

In [14]:
edge_counts = results.sum()
most_frequent_edge = edge_counts.idxmax()
max_count = edge_counts.max()

print (f"The most frequent edge is {most_frequent_edge} with {max_count} occurrences.")

The most frequent edge is ('CCDC17', 'SCGB1A1') with 9.0 occurrences.
